In [1]:
from time import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution1D, Dropout, MaxPooling1D, Conv1D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import TensorBoard

from numpy.random import seed
from tensorflow import set_random_seed
seed(3141593)
set_random_seed(3141593)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def encode(train, test):
    label_encoder = LabelEncoder().fit(train.species)
    labels = label_encoder.transform(train.species)
    classes = list(label_encoder.classes_)

    train = train.drop(['species', 'id'], axis=1)
    test = test.drop('id', axis=1)

    return train, labels, test, classes

train, labels, test, classes = encode(train, test)

# standardize train features
scaler = StandardScaler().fit(train.values)
scaled_train = scaler.transform(train.values)
scaled_test = scaler.transform(test)

# split train data into train and validation
sss = StratifiedShuffleSplit(test_size=0.1, random_state=123)
for train_index, valid_index in sss.split(scaled_train, labels):
    X_train, X_valid = scaled_train[train_index], scaled_train[valid_index]
    y_train, y_valid = labels[train_index], labels[valid_index]

feature_number = 64
class_number = len(classes)

y_train = np_utils.to_categorical(y_train, class_number)
y_valid = np_utils.to_categorical(y_valid, class_number)    

# reshape train data
X_train_r = np.zeros((len(X_train), feature_number, 3))
X_train_r[:, :, 0] = X_train[:, :feature_number]
X_train_r[:, :, 1] = X_train[:, feature_number:128]
X_train_r[:, :, 2] = X_train[:, 128:]

# reshape validation data
X_valid_r = np.zeros((len(X_valid), feature_number, 3))
X_valid_r[:, :, 0] = X_valid[:, :feature_number]
X_valid_r[:, :, 1] = X_valid[:, feature_number:128]
X_valid_r[:, :, 2] = X_valid[:, 128:]

# reshape y train data
y_train_all = [[0 for i in range(99)] for j in range(990)]
for i, label in enumerate(labels):
    y_train_all[i][label] = 1
y_train_all = np.array(y_train_all)

# reshape test data
test_r = np.zeros((len(scaled_test), feature_number, 3))
test_r[:, :, 0] = scaled_test[:, :feature_number]
test_r[:, :, 1] = scaled_test[:, feature_number:128]
test_r[:, :, 2] = scaled_test[:, 128:]

model = Sequential()
model.add(Conv1D(input_shape=(64, 3), filters=1024, kernel_size=6))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(rate = 1 - 0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(class_number))
model.add(Activation('softmax'))

adam = Adam(lr=1e-4, decay=1e-5)

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

model.fit(X_train_r, y_train, epochs=100, validation_data=(X_valid_r, y_valid), batch_size=32, callbacks = [tensorboard])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 891 samples, validate on 99 samples
Epoch 1/100
891/891 [==============================] - 5s 6ms/step - loss: 4.2281 - acc: 0.1167 - val_loss: 3.2018 - val_acc: 0.4444
Epoch 2/100
891/891 [==============================] - 2s 2ms/step - loss: 2.1409 - acc: 0.6453 - val_loss: 1.0596 - val_acc: 0.8889
Epoch 3/100
891/891 [==============================] - 2s 2ms/step - loss: 0.6252 - acc: 0.8956 - val_loss: 0.3817 - val_acc: 0.9394
Epoch 4/100
891/891 [==============================] - 2s 2ms/step - loss: 0.2158 - acc: 0.9686 - val_loss: 0.1811 - val_acc: 0.9798
Epoch 5/100
891/891 [==============================] - 2s 2ms/step - loss: 0.0917 - acc: 0.9899 - val_loss: 0.0906 - val_acc: 1.0000
Epoch 6/100
891/891 [==============================] - 

Epoch 54/100
891/891 [==============================] - 2s 2ms/step - loss: 4.3654e-04 - acc: 1.0000 - val_loss: 0.0117 - val_acc: 1.0000
Epoch 55/100
891/891 [==============================] - 2s 2ms/step - loss: 4.1024e-04 - acc: 1.0000 - val_loss: 0.0110 - val_acc: 1.0000
Epoch 56/100
891/891 [==============================] - 2s 2ms/step - loss: 4.1771e-04 - acc: 1.0000 - val_loss: 0.0135 - val_acc: 1.0000
Epoch 57/100
891/891 [==============================] - 2s 2ms/step - loss: 3.6065e-04 - acc: 1.0000 - val_loss: 0.0132 - val_acc: 1.0000
Epoch 58/100
891/891 [==============================] - 2s 2ms/step - loss: 3.7316e-04 - acc: 1.0000 - val_loss: 0.0145 - val_acc: 1.0000
Epoch 59/100
891/891 [==============================] - 2s 2ms/step - loss: 3.3492e-04 - acc: 1.0000 - val_loss: 0.0135 - val_acc: 1.0000
Epoch 60/100
891/891 [==============================] - 2s 2ms/step - loss: 3.6405e-04 - acc: 1.0000 - val_loss: 0.0114 - val_acc: 1.0000
Epoch 61/100
891/891 [============

In [2]:
pred = model.predict(test_r)
sample_sub = pd.read_csv('sample_submission.csv')
submission = pd.DataFrame(data = pred)
submission.columns = sample_sub.columns[1:]
submission['id'] = sample_sub['id']
submission.to_csv("submission.csv", sep=',', index = False)